In [21]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder

from xgboost import XGBClassifier

import pickle as pk

# jtplot.style(figsize=(15, 9))

In [2]:
! wget 'https://www.dropbox.com/scl/fi/0iit58hu7dcwzc0ecpz2i/SHR65_23.csv?rlkey=gad3m6vkqt23iezl2sxny3euo&st=pug1groy&dl=1'

--2024-11-14 18:37:20--  https://www.dropbox.com/scl/fi/0iit58hu7dcwzc0ecpz2i/SHR65_23.csv?rlkey=gad3m6vkqt23iezl2sxny3euo&st=pug1groy&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd9f0c10fd6e74af0e545854682.dl.dropboxusercontent.com/cd/0/inline/Cea1dK0PSUikflItLRuw--IKOcujbkqm_z1HrkPS_HfuNI_qLZl0tQK2IoTTv8mkDttGMPdz40-rxDPjmxkbRJ1417aqNQ8wVfmEHgglCmZ1JeHur-x7htJZ9AAFhCUs0jqvkhk1ZABZj1SUR9mFq578/file?dl=1# [following]
--2024-11-14 18:37:21--  https://ucd9f0c10fd6e74af0e545854682.dl.dropboxusercontent.com/cd/0/inline/Cea1dK0PSUikflItLRuw--IKOcujbkqm_z1HrkPS_HfuNI_qLZl0tQK2IoTTv8mkDttGMPdz40-rxDPjmxkbRJ1417aqNQ8wVfmEHgglCmZ1JeHur-x7htJZ9AAFhCUs0jqvkhk1ZABZj1SUR9mFq578/file?dl=1
Resolving ucd9f0c10fd6e74af0e545854682.dl.dropboxusercontent.com (ucd9f0c10fd6e74af0e545854682.dl.dropboxusercontent.c

In [11]:
map_data = pd.read_csv('SHR65_23.csv')
map_data.drop('ID', axis=1, inplace=True)
map_data.head()

,CNTYFIPS,Ori,State,Agency,Agentype,Source,Solved,Year,Month,Incident,...,OffRace,OffEthnic,Weapon,Relationship,Circumstance,Subcircum,VicCount,OffCount,FileDate,MSA
0,"Anchorage, AK",AK00101,Alaska,Anchorage,Municipal police,FBI,Yes,1976,March,1,...,Black,Unknown or not reported,"Handgun - pistol, revolver, etc",Relationship not determined,Other arguments,NaN,0,0,30180.0,"Anchorage, AK"
1,"Anchorage, AK",AK00101,Alaska,Anchorage,Municipal police,FBI,Yes,1976,April,1,...,White,Unknown or not reported,"Handgun - pistol, revolver, etc",Girlfriend,Other arguments,NaN,0,0,30180.0,"Anchorage, AK"
2,"Anchorage, AK",AK00101,Alaska,Anchorage,Municipal police,FBI,Yes,1976,June,1,...,Black,Unknown or not reported,"Handgun - pistol, revolver, etc",Stranger,Other,NaN,0,0,30180.0,"Anchorage, AK"
3,"Anchorage, AK",AK00101,Alaska,Anchorage,Municipal police,FBI,Yes,1976,June,2,...,White,Unknown or not reported,"Handgun - pistol, revolver, etc",Other - known to victim,Other arguments,NaN,0,0,30180.0,"Anchorage, AK"
4,"Anchorage, AK",AK00101,Alaska,Anchorage,Municipal police,FBI,Yes,1976,July,1,...,American Indian or Alaskan Native,Unknown or not reported,Knife or cutting instrument,Brother,Other arguments,NaN,0,0,30180.0,"Anchorage, AK"


In [12]:
useless_clmns = ['FileDate', 'Source', 'ActionType', 'Ori', 'Subcircum'] # 'Solved'
map_data.drop(useless_clmns, axis=1, inplace=True)

In [13]:
trans_clmns = map_data.dtypes[map_data.dtypes != 'int64'].index.to_list()

In [14]:
yes = map_data[(map_data['Solved']=='Yes')
               & ~(map_data['OffSex']=='Unknown')
               & ~(map_data['OffRace']=='Unknown')
              ].index
no = map_data[map_data['Solved']=='No'].index # test

In [7]:
yes.shape, no.shape

((620281,), (262179,))

In [ ]:
# Relationship not determined / Circumstances undetermined

In [17]:
le = LabelEncoder()
for i in trans_clmns:
    map_data[i] = le.fit_transform(map_data[i])

In [18]:
yes = map_data.loc[yes]
no = map_data.loc[no]

In [19]:
X = yes.drop('Relationship', axis=1)
y = yes['Relationship']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [25]:
bst = XGBClassifier(n_estimators=500, learning_rate=0.1)
bst.fit(X_train, y_train)

bst.score(X_test, y_test)

In [ ]:
with open('ml/relationship_RF.pk', 'wb') as f:
    pk.dump(_model, f, protocol=5)